In [3]:
import darknet
import darknet_images
import time
"""
load model description and weights from config files
args:
    config_file (str): path to .cfg model file
    data_file (str): path to .data model file
    weights (str): path to weights
returns:
    network: trained model
    class_names
    class_colors
"""
network, class_names, class_colors = darknet.load_network(
        '/home/jetbot/g031r066/darknet/custom/yolov4-tiny-custom.cfg', 
        '/home/jetbot/g031r066/darknet/custom/custom.data',  
        '/home/jetbot/g031r066/darknet/custom/backup/yolov4-tiny-custom_final.weights'
    )

ModuleNotFoundError: No module named 'darknet'

In [1]:
import sys
sys.path

['/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '',
 '/home/jetbot/.local/lib/python3.6/site-packages',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/torchvision-0.4.0a0+d31eafa-py3.6-linux-aarch64.egg',
 '/usr/local/lib/python3.6/dist-packages/Adafruit_SSD1306-1.6.2-py3.6.egg',
 '/usr/local/lib/python3.6/dist-packages/Adafruit_MotorHAT-1.4.0-py3.6.egg',
 '/usr/local/lib/python3.6/dist-packages/Adafruit_GPIO-1.0.4-py3.6.egg',
 '/usr/local/lib/python3.6/dist-packages/spidev-3.4-py3.6-linux-aarch64.egg',
 '/usr/local/lib/python3.6/dist-packages/Adafruit_PureIO-1.0.4-py3.6.egg',
 '/usr/local/lib/python3.6/dist-packages/jetbot-0.4.0-py3.6.egg',
 '/usr/lib/python3/dist-packages',
 '/usr/lib/python3.6/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/home/jetbot/.ipython']

In [22]:
images_path = 'custom/test_traffic_light_red.jpg'
images = darknet_images.load_images(images_path)

image_name = images[0]
prev_time = time.time()
thresh = .30
image, detections = darknet_images.image_detection(image_name, network, class_names, class_colors, thresh)

darknet.print_detections(detections, True)  # True: 各物体の座標・幅・高さを表示
fps = int(1/(time.time() - prev_time))
print("FPS: {}".format(fps))
print("Predicted in {} seconds".format((time.time() - prev_time)))


Objects:
Traffic light: 58.88%    (left_x: 259   top_y:  241   width:   88   height:  218)
FPS: 9
Predicted in 0.10379338264465332 seconds


In [23]:
from IPython.display import display
import ipywidgets

image_widget = ipywidgets.Image(format='jpg', width=400, height=400)
display(image_widget)

Image(value=b'', format='jpg', height='400', width='400')

In [24]:
import cv2

# drawn_imagesの型はnumpy.ndarray、ipywidgetsはbytesなので表示出来る型に変換する必要がある
bytes_images = cv2.imencode(".jpg", image, (cv2.IMWRITE_JPEG_QUALITY, 100))[1].tobytes()  # [0]:変換が成功したかどうかのbool値 [1]:エンコードされたバイト列を tuple で返す。
image_widget.value = bytes_images

In [25]:
label_list=[]
confidence_list=[]
bbox_list=[]
for label, confidence, bbox in detections: 
    label_list.append(label)
    confidence_list.append(confidence)
    bbox_list.append(bbox)

print(label_list)   
print(confidence_list)
print(*bbox_list,sep='\n')

['Traffic light']
['58.88']
(259.0245666503906, 240.69375610351562, 87.5177993774414, 218.49827575683594)


In [26]:
# 信号機部分を切り出し
from decimal import Decimal, ROUND_HALF_UP

i=0
xmin, ymin, xmax, ymax = darknet.bbox2points(bbox_list[i])
print(xmin, ymin, xmax, ymax)

215 131 303 350


In [30]:
cv2.imwrite("normal_image.jpg",image)
trimmed_image = image[ymin+1:ymax-1, xmin+1:xmax-1]  # bboxの枠線が映らないようにちょっと狭めに切り取る
cv2.imwrite("trimmed_image.jpg", trimmed_image)

True

In [31]:
# 色基準で2値化する。
hsv = cv2.cvtColor(trimmed_image, cv2.COLOR_BGR2HSV)

# HSV平均値を取得
# flattenで一次元化しmeanで平均を取得 
h = hsv.T[0].flatten().mean()
s = hsv.T[1].flatten().mean()
v = hsv.T[2].flatten().mean()

# HSV平均値を出力
# uHeは[0,179], Saturationは[0,255]，Valueは[0,255]
print("Hue: %.2f" % (h))
print("Salute: %.2f" % (s))
print("Value: %.2f" % (v))

# 信号機の色を判別
# 参考1 : https://algorithm.joho.info/programming/python/opencv-color-detection/
# 参考2 : https://tomosoft.jp/design/?p=44101
if((h>=0 and h<=30) or ((h>=150 and h<=179))):
    print("Traffic_light : red")
elif(h>30 and h<=150):  # 緑と青は取り敢えず一緒にした
    print("Traffic_light : green")
else:
    print("Traffic_light : none")

Hue: 96.80
Salute: 149.89
Value: 122.44
Traffic_light : green
